<a href="https://colab.research.google.com/github/rhodeskirkpatrick1998/WrittenWork_Portfolio/blob/main/Reuters_News_Keras_Neural_Network_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data:

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

Goal:

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

In [ ]:
!pip install tensorflow==2.11.0rc0

In [ ]:
import tensorflow as tf
if not tf.__version__ == '2.11.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the num_words key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a Dense layer. For this we use a Tokenizer from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length max_words=1000. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

label encoding:

Use to_categorical, to transform both y_train and y_test into one-hot encoded vectors of length num_classes:

In [ ]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Model definition:

Next, initialise a Keras Sequential model and add three layers to it:

Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
Layer: Add a *Dropout* layer with dropout rate of 50%.
Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [ ]:
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(1000,)))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

model compilation:

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric.

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model training and evaluation:

Next, define the batch_size for training as 32 and train the model for 5 epochs on x_train and y_train by using the fit method of your model. Then calculate the score for your trained model by running evaluate on x_test and y_test with the same batch size as used in fit.

In [ ]:
batch_size = 32
model.fit(x_train,y_train,batch_size=batch_size,epochs=5,validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test,batch_size=32)

In [ ]:
score[1]

0.8032057285308838

In [ ]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

In [ ]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "rhodesjkirkpatrick@gmail.com"
token = "3sT6FT6yOufi5iV4"


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))